## Fine-tuning a masked language model

- For this exercise, we will just DistilBERT as an example 

In [9]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
import torch
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

In [10]:
import collections
import numpy as np
from transformers import default_data_collator
from transformers import TrainingArguments
from transformers import Trainer

In [11]:
import config
import os

- Basic explorations

In [12]:
model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


- Try the masked language prediction task

In [13]:
text = "This is a great [MASK]."

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


### Load dataset for finetuning language model 
- here we will sue imdb for fintuning purpose 

In [14]:
imdb_dataset = load_dataset("imdb")
imdb_dataset

Reusing dataset imdb (/home/chengyu/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

- some basic data exploration 

In [15]:
sample = imdb_dataset["train"].shuffle(seed=42).select(range(2))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")

Loading cached shuffled indices for dataset at /home/chengyu/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-8a9e43a6ac4acdff.arrow



'>>> Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'
'>>> Label: 1'

'>>> Review: This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stu

- if your deomain data has a lot domain specific terms, it is a good idea to do an vocabulary expansion 

In [16]:
from transformers_domain_adaptation import VocabAugmentor

In [17]:
## original vocabulary size 
tokenizer.vocab_size

30522

In [18]:
## set a new vocab size limit  
target_vocab_size = 31_000  # len(tokenizer) == 30_522
augmentor = VocabAugmentor(
    tokenizer=tokenizer,   ## the domain_adaptation package require tokenizer to be a older version. may need another environment
    cased=False,    # change this based on your tokenizer
    target_vocab_size=target_vocab_size
)

# Obtain new domain-specific terminology based on the fine-tuning corpus
new_tokens = augmentor.get_new_tokens(imdb_dataset["train"]['text'])

TypeError: Can't convert <tokenizers.trainers.WordPieceTrainer object at 0x7ffa12816e90> to Sequence

In [11]:
## depends on your use case, maybe you want to do some cleaning on the new vocabularies 
print(len(new_tokens))
print(new_tokens[:100])

478
['/><', '/>', '.<', '),', ').', '--', '",', '".', '."', ',"', '\x96', '..', ').<', '!<', '....', '...<', '.)', '!!', 'dialog', "',", '?<', '!"', 'believable', 'imdb', "'.", '!!!', 'cheesy', '/>"', 'nudity', '!)', 'spoilers', '?"', '("', '")', 'slasher', '?)', '".<', 'laughable', 'spoiler', 'likable', 'mediocre', 'cliché', 'funniest', 'flicks', ".'", 'thats', '.,', 'cgi', 'redeeming', 'wonderfully', 'keaton', 'watchable', 'clichés', '.....', 'pretentious', ':<', '."<', '\x97', '**', '/>-', 'unfunny', 'gags', ');', 'corny', '***', '****', 'cant', 'brilliantly', '!).', 'gratuitous', 'crappy', 'gory', 'underrated', '??', '---', 'contrived', 'unrealistic', 'depressing', 'amateurish', 'tedious', 'lousy', '/>*', ",'", 'columbo', '):', 'liners', 'forgettable', 'pacino', 'uninteresting', 'atrocious', '???', 'gritty', 'lugosi', 'unconvincing', 'suspenseful', '?!', 'subtitles', ')<', 'melodrama', '!!!!']


- ### Now very importtant, add new tokens to original tokenizer and resized model embeding layers size 

In [12]:
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(31000, 768)

### Preprocessing the data

- For both auto-regressive and masked language modeling, a common preprocessing step is to concatenate all the examples and then split the whole corpus into chunks of equal size. This is quite different from our usual approach, where we simply tokenize individual examples. Why concatenate everything together? The reason is that individual examples might get truncated if they’re too long, and that would result in losing information that might be useful for the language modeling task!

- So to get started, we’ll first tokenize our corpus as usual, but without setting the truncation=True option in our tokenizer. We’ll also grab the word IDs if they are available ((which they will be if we’re using a fast tokenizer, as described in Chapter 6), as we will need them later on to do whole word masking. 

In [13]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        ## get world id for whole word masking 
    return result

# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
#tokenized_datasets

Token indices sequence length is longer than the specified maximum sequence length for this model (577 > 512). Running this sequence through the model will result in indexing errors


- Now group them all together and split the result into chunks.

In [14]:
#firt check tokenizer max length
tokenizer.model_max_length

512

In [15]:
#in order to run our experiments on GPUs like those found on Google Colab, we’ll pick something a bit smaller that can fit in memory:
chunk_size = 128

In [16]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 197'
'>>> Review 1 length: 577'
'>>> Review 2 length: 195'


In [17]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys() # for each key, merge all lists
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 969'


In [18]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 73'


- for the last chunk, it will be smaller than everything else, we will just drop it 
- now put everything together in one fucntion 

In [19]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column, for MLM, we put labels as inputs ids 
    result["labels"] = result["input_ids"].copy()
    return result

In [20]:
## process our data with grouping function
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 61364
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 60020
    })
    unsupervised: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 123243
    })
})

### Insert [MASK] tokens at random positions in the inpouts; we will use a special data collator so that the masking can be apply on the fly
- Transformers comes prepared with a dedicated DataCollatorForLanguageModeling for just this task. We just have to pass it the tokenizer and an mlm_probability argument that specifies what fraction of the tokens to mask. We’ll pick 15%, which is the amount used for BERT and a common choice in the literature:

In [21]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

- let's take a look at what this data collector do 

In [22]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples: 
    _ = sample.pop("word_ids")  ## data collator does not take word_ids as input
    
for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'") ## it will just insert mask token in random places 


'>>> [CLS] [MASK] [MASK]well high is a sat comedy. it ran at the same time as some other programs about sch ool [MASK], such as reject teachers ". [MASK] 35 years in [MASK] teaching [MASK] lead me to believe that bromwell [MASK]'s satire is much closer to rea lity than is " teachers ". the scramble to survive financially, [MASK] insightful studentsesis can see right through their pathetic [MASK]'pomp, the pettiness of parody [MASK] situation [MASK] all remind me of the sch ools i knew and their students. when i saw the episode in which a student repeatedly tried [MASK] burn down the sch ool, i immediately'

'>>> recalled.. [MASK]..... at.. woodrow...... [MASK]. a [MASK] line [MASK] inspector : i'm here to sack one of your teachers. student [MASK] [MASK] to [MASK]mwell high. i expect [MASK] many adults of [MASK] age think that bromwell [MASK] is far fetched. what a pity that [MASK] i sn't! [SEP] [CLS] homeles sn ess ( or houseles [MASK] ess as george [MASK]in stated ) has been an issue

- One side effect of random masking is that our evaluation metrics will not be deterministic when using the Trainer, since we use the same data collator for the training and test sets. We’ll see later, when we look at fine-tuning with 🤗 Accelerate, how we can use the flexibility of a custom evaluation loop to freeze the randomness.

### Whold word Masking

- When training models for masked language modeling, one technique that can be used is to mask whole words together, not just individual tokens. This approach is called whole word masking. If we want to use whole word masking, we will need to build a data collator ourselves. A data collator is just a function that takes a list of samples and converts them into a batch, so let’s do this now! We’ll use the word IDs computed earlier to make a map between word indices and the corresponding tokens, then randomly decide which words to mask and apply that mask on the inputs. Note that the labels are all -100 except for the ones corresponding to mask words.

In [23]:
wwm_probability = 0.2

In [24]:
feature = lm_datasets["train"][0]

In [25]:
def whole_word_masking_data_collator(features):
    for feature in features: 
        word_ids = feature.pop("word_ids")  ## data collator does not take word_ids
 
        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)  ## create a whold word to token ids map

        mask = np.random.binomial(1, wwm_probability, (len(mapping),)) ## create a 0 , 1 list with x prob to be 1
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels) 
        for word_id in np.where(mask)[0]: ## loop through all masked idx 
            word_id = word_id.item()
            for idx in mapping[word_id]: ## use mapping to get token indexes 
                new_labels[idx] = labels[idx]  ## in lables, masked token will be actual token index
                input_ids[idx] = tokenizer.mask_token_id   ## masked tokens will be replaced with mask token id
    
    return default_data_collator(features)

- try it on some examples

In [26]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] bromwell high is a cartoon comedy. it ran at the same time as some other [MASK] about sch ool [MASK], such as " teachers ". my [MASK] years in the [MASK] [MASK] lead me to [MASK] [MASK] bromwell high's satire is much closer [MASK] rea [MASK] [MASK] than [MASK] " teachers ". [MASK] [MASK] to survive financially, the insightful [MASK] [MASK] can see right through [MASK] [MASK] [MASK]'pomp, the pettiness of [MASK] whole situation, all remind me [MASK] the sch ools [MASK] knew and their [MASK] [MASK] when [MASK] saw the episode [MASK] which a [MASK] repeatedly tried to burn down [MASK] sch ool, i immediately'

'>>> [MASK]...... [MASK]. at.... [MASK].... high. a [MASK] line : inspector [MASK] i'[MASK] here to sack one of [MASK] teachers. [MASK] : welcome [MASK] bromwell high. i expect [MASK] many adults of my age [MASK] [MASK] [MASK] [MASK] [MASK] high is far fetched [MASK] what [MASK] pity that it i [MASK]'[MASK]! [SEP] [CLS] homeles sn ess ( [MASK] houseles [MASK] [MASK] [MASK

### Now we can strat training

- for demo purpose, we will first downsample our data 

In [27]:
train_size = 10_000
test_size = int(0.5 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 5000
    })
})

-Here we tweaked a few of the default options, including logging_steps to ensure we track the training loss with each epoch. We’ve also used fp16=True to enable mixed-precision training, which gives us another boost in speed. By default, the Trainer will remove any columns that are not part of the model’s forward() method. This means that if you’re using the whole word masking collator, you’ll also need to set remove_unused_columns=False to ensure we don’t lose the word_ids column during training.

In [28]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]
model_out_dir = os.path.join(config.data_folder,'MLM','models',"{}-finetuned-imdb".format(model_name))

training_args = TrainingArguments(
    output_dir=model_out_dir,
    overwrite_output_dir=True,
    num_train_epochs = 10,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #push_to_hub=False,  ## just run locally for now 
    fp16=True,
    logging_steps=logging_steps,
)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
## see the original perplexity before training
trainer.evaluate()

/home/chengyu/anaconda3/envs/gpt3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


wandb: Currently logged in as: chuang16 (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'eval_loss': 4.292020797729492,
 'eval_runtime': 14.3694,
 'eval_samples_per_second': 347.96}

In [31]:
trainer.train()

/home/chengyu/anaconda3/envs/gpt3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/chengyu/anaconda3/envs/gpt3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,3.036516,12.924100,386.874000
2,3.271300,2.891255,12.865800,388.627000
3,3.271300,2.783624,12.811100,390.288000
4,2.919400,2.738822,13.129400,380.826000
5,2.919400,2.704603,12.909500,387.311000
6,2.797900,2.669362,13.031000,383.701000
7,2.797900,2.646639,13.373800,373.867000
8,2.752300,2.655278,13.401100,373.103000
9,2.752300,2.636327,13.148600,380.268000
10,2.731200,2.632193,13.255300,377.208000


/home/chengyu/anaconda3/envs/gpt3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=790, training_loss=2.8925721663462967, metrics={'train_runtime': 748.1911, 'train_samples_per_second': 1.056, 'total_flos': 5172719001600000, 'epoch': 10.0})